# **STINTSY Machine Learning Project: Pumpkin Seeds Dataset**

**STINTSY S11 - Milky Way** \
*Group Members:*
- Gutierrez, Mark Daniel
- Refuerzo, Lloyd Dominic
- Romblon, Kathleen Mae
- Stinson, Audrey Lauren

## **1** | **Introduction**

The pumpkin plant belongs to the Cucurbitaceae family and has seasonal varieties. Confectionery pumpkins, grown in Turkey, are usually produced from the pumpkin species, *Cucurbita pepo* L and sometimes from the *Cucurbita moschata* Duchesne type. Pumpkin seeds are considered as important for human health because it contains 37 percent of carbohydrate, 35 percent to 40 percent of fat and protein along with calcium, potassium, phosphorus, magnesium, iron, and zinc. Pumpkins are divided into many types, and one of these species is known as “Urgup Sivrisi”. Urgup Sivrisi is a type of pumpkin seed that has a long, white, very bright, thin, and hardly distinguishable shell with a pointed tip. The other type of pumpkin seeds is “Cercevelik”. It is a particular species grown in Turkey, Nevsehir, Karacaoren, and known as “Topak” in Turkey. <span style="color:#42adf5">(*taken directly from* Details *section of the Pumpkin Seeds Dataset pdf*)</span>

The target task for this dataset is to correctly classify whether an image of a pumpkin seed is of the species type "Urgup Sivrisi" or "Cercevelik". The dataset then offers a <span style="color:#f5b942">classification problem</span> that the group will address through the use of various machine learning models, namely **k-Nearest Neighbors**, **Decision Trees**, and **Logistic Regression**.

## **2** | **About the Dataset**

The dataset, collected by Koklu et al. (2021), contains extracted features from 2500 images of two varieties of pumpkin seeds, Urgup Sivrisi and Cercevelik. These images were taken inside a product shooting box to prevent shadows from showing if light from outside of the box were to get in. To process the original RGB images, they were converted to gray-toned images, and then to binary images to simplify the value of each pixel in the image. As the RGB images will be converted to binary images for the image processing part, the shadows can make the acquired size and shape of the seed appear smaller.

From the image binarization of each of the 2500 images, 12 features were extracted for each instance. The extracted features are based on the shape of the pumpkin seeds, where each pixel in the image was calculated while considering the values of other nearby pixels.

As such, the extracted features are as follows:

1. <span style="color:#f5b942">Area:</span> Number of pixels within the borders of a pumpkin seed
2. <span style="color:#f5b942">Perimeter:</span> Circumference in pixels of a pumpkin seed
3. <span style="color:#f5b942">Major Axis Length:</span> Large axis distance of a pumpkin seed
4. <span style="color:#f5b942">Minor Axis Length:</span> Small axis distance of a pumpkin seed
5. <span style="color:#f5b942">Convex Area:</span> Number of pixels of the smallest convex shell at the region formed by the pumpkin seed
6. <span style="color:#f5b942">Equiv Diameter:</span> Computed as $\sqrt{4a/\pi}$, where $a$ is the area of the pumpkin seed
7. <span style="color:#f5b942">Eccentricity:</span> Eccentricity of a pumpkin seed
8. <span style="color:#f5b942">Solidity:</span> Convex condition of the pumpkin seeds
9. <span style="color:#f5b942">Extent:</span> Ratio of a pumpkin seed area to the bounding box pixels
10. <span style="color:#f5b942">Roundness:</span> Ovality of pumpkin seeds without considering the distortion of the edges
11. <span style="color:#f5b942">Aspect Ratio:</span> Aspect ratio of the pumpkin seeds
12. <span style="color:#f5b942">Compactness:</span> Proportion of the area of the pumpkin seed relative to the area of the circle with the same circumference

## **3** | **List of Requirements**

The following cell imports the libraries needed for setting up the notebook:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

## **4** | **Data Preprocessing**

### *Importing the dataset*

For the following cell, we will be using the `read_csv` function to import the pumpkin seeds dataset to our notebook.

In [ ]:
data = pd.read_csv('pumpkin_seeds.csv')

To check that we have imported the data, we can take a look at the first and last 10 instances in the dataset.

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
# (no. of instances, no. of columns [features + target label])
data.shape

The dataset consists of 13 columns, where the first 12 columns are the input features and the last column is the target label. There are 2500 instances in total, and the shape of the dataset is `(2500, 13)`.

Optionally, we can see the statistical summary of the dataset by calling the `describe()` function.

In [ ]:
data.describe()

### *Data Cleaning*

As we can see, one of the features, `Aspect_Ration`, is spelled incorrectly. To fix this, we will rename that column to `Aspect_Ratio`.

In [ ]:
#renaming and reformatting the features
data.columns = ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Convex_Area', 'Equiv_Diameter',
                'Eccentricity', 'Solidity', 'Extent', 'Roundness', 'Aspect_Ratio', 'Compactness', 'Class']

The class labels can be renamed so that it only includes letters from the English alphabet, and this will be done by running the following cell block.

In [ ]:
data['Class'] = data['Class'].str.strip().str.title().replace({'Çerçevelik': 'Cercevelik'})
data['Class'] = data['Class'].str.strip().str.title().replace({'Ürgüp Sivrisi': 'Urgup Sivrisi'})
#data.info()
#data.hist(figsize=(12,12))
#plt.show()

In continuation, we can check if there are other representations of the Class column by calling the `unique()` function on it. Since there are only two unique values, we do not need to make any changes for this column for now.

In [ ]:
data['Class'].unique()

Using the `info()` function, we can check any feature with incorrect datatype. If there are inconsistencies with the datatype, it is likely to be assigned an `object` datatype. It should also be noted that for features that we would usually assume to have a `string` datatype, it is possible that they have `object` datatype.

In [ ]:
data.info()

Only one of these features have the `object` datatype assigned to it, which is the `Class` column. However, since we have already queried its unique values in the previous section, we know that there shouldn't be inconsistencies in this column, so we can keep this column as is.

Next, we need to check if there are any missing values or instances where a default value has been assigned. In a pandas dataframe, these are usually represented as `None` or `NaN`, so we can query for any null values in our dataset. Additionally, we can also check if there are any duplicated instances.

In [ ]:
display(data.isnull().sum())
display(data.duplicated().sum())

Now that we have finished checking the data, we can now split the data into features (`X`) and the target label (`y`).

In [ ]:
#split features and label
X = data.drop(columns=['Class','Convex_Area','Equiv_Diameter']).values
y = data['Class'].values

Some models may require that our features are normalized, so we'll define a normalized X variable using the `MinMaxScaler` library's `fit_transform()` function.

In [ ]:
#normalize
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

Our class labels can be represented numerically. To do this, we use the `LabelEncoder` library's `fit_transform()` function on our target label `y` to represent **Cercevelik** as class `0` and **Urgup Sivrisi** as class `1`.

In [ ]:
#Encode Cercevelik as 0, Urgup Sivrisi as 1
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

#Check if y is properly transformed
np.unique(y)

Similarly, for the purposes of our exploratory data analysis, we will also transform the `Class` column in our dataframe so that **Cercevelik** is represented as `0` and **Urgup Sivrisi** is represented as class `1`. We will be doing this in a temporary dataframe so as not to affect the original dataframe with the named labels.

In [ ]:
temp_df = data

# temp_df['Class'] = label_encoder.fit_transform(temp_df['Class'])

temp_df['Class'].unique()

## **5** | **Exploratory Data Analysis**

There are some things that we can take into consideration when trying to classify a pumpkin seed. As this is a dataset that contains morphological features of the two specific species of pumpkin seeds, we can explore the features that represent the size and the shape of the seed. Some of the EDA questions that we have come up with are listed in the following:

*General:*
- How much of the data are Cercevelik pumpkin seeds and Urgup Sivrisi pumpkin seeds?

*Size:*
- What are the features of the seed that has the smallest perimeter? The largest?
    - What is the average perimeter of each of the pumpkin seeds?
- What are the features of the most narrow seed of each pumpkin seed? The widest?
    - What is the average width of each of the pumpkin seeds?
- What are the features of the shortest seed of each pumpkin seed? The longest?
    - What is the average length of each of the pumpkin seeds?

*Shape:*
- What are the features of the most circular seed for each of the pumpkin seed species? The most elongated?
    - What is the average eccentricity of each of the pumpkin seed species?
- What are the features of the least round seed for each of the pumpkin seed species? The most round?
    - What is the average roundness of each of the pumpkin seed species?
- <span style="color:red">Convex area</span>

### How much of the data are Cercevelik pumpkin seeds and Urgup Sivrisi pumpkin seeds?

To get the count of each of the pumpkin seed species in the dataset, we can group our dataset by `Class` and call the `size()` function on it.

In [ ]:
# Count of each pumpkin seed
seed_counts = temp_df.groupby('Class').size()

seed_counts

As we can see, there are 1300 Cercevelik pumpkin seeds and 1200 Urgup Sivrisi pumpkin seeds in the dataset. Since we have grouped our dataset by `Class`, we can assign these groups to their own variables so that we can further explore our data. We would also need to reset the index so that both of the new dataframes start at index `0`.

In [ ]:
# Group according to class (Cercevelik or Urgup Sivrisi)
groups = temp_df.groupby('Class')

# Assign to their own variables to check in further EDA questions
cercevelik_seeds, urgup_seeds = groups.get_group("Cercevelik"), groups.get_group("Urgup Sivrisi")

In [ ]:
# Reset index for Cercevelik
cercevelik_seeds = cercevelik_seeds.reset_index()

# Drop index column
cercevelik_seeds.drop(columns="index", axis=1, inplace=True)

In [ ]:
# Reset index for Urgup Sivrisi
urgup_seeds = urgup_seeds.reset_index()

# Drop index column
urgup_seeds.drop(columns="index", axis=1, inplace=True)

We can check the new dataframes in the following cells.

In [ ]:
cercevelik_seeds

In [ ]:
urgup_seeds

The following plot shows how many seeds there are in each of the species in graph form, and compares them side by side.

In [ ]:
g = sns.catplot(data=temp_df,x='Class',kind='count')
g.set_axis_labels("", "Number of seeds")
g.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

The following few questions are questions created for probing on the **Size** of the seeds. The features that we think are relevant to learning about the difference in size between the two pumpkin seed species are `Perimeter`, `Minor Axis Length`, and `Major Axis Length`.

### What are the features of the seed that has the smallest perimeter?

In [ ]:
# Cercevelik min perimeter
cercevelik_perim_min = cercevelik_seeds['Perimeter'].min()
# Index
cercevelik_perim_min_idx = cercevelik_seeds['Perimeter'].idxmin()

print("Cercevelik seed with smallest perimeter =", cercevelik_perim_min) # Value
cercevelik_seeds.iloc[[cercevelik_perim_min_idx]] # Instance

In [ ]:
# Urgup sivrisi min perimeter
urgup_perim_min = urgup_seeds['Perimeter'].min()
# Index
urgup_perim_min_idx = urgup_seeds['Perimeter'].idxmin()

print("Urgup Sivrisi seed with smallest perimeter =", urgup_perim_min) # Value
urgup_seeds.iloc[[urgup_perim_min_idx]] # Instance

### The largest?

In [ ]:
# Cercevelik max perimeter
cercevelik_perim_max = cercevelik_seeds['Perimeter'].max()
# Index
cercevelik_perim_max_idx = cercevelik_seeds['Perimeter'].idxmax()

print("Cercevelik seed with largest perimeter =", cercevelik_perim_max) # Value
cercevelik_seeds.iloc[[cercevelik_perim_max_idx]] # Instance

In [ ]:
# Urgup sivrisi max perimeter
urgup_perim_max = urgup_seeds['Perimeter'].max()
# Index
urgup_perim_max_idx = urgup_seeds['Perimeter'].idxmax()

print("Urgup Sivrisi seed with largest perimeter =", urgup_perim_max) # Value
urgup_seeds.iloc[[urgup_perim_max_idx]] # Instance

### What is the average perimeter of each of the pumpkin seeds?

In [ ]:
cercevelik_perim_avg = cercevelik_seeds['Perimeter'].mean()

print("Cercevelik seed average perimeter =", cercevelik_perim_avg)

In [ ]:
urgup_perim_avg = urgup_seeds['Perimeter'].mean()

print("Urgup Sivrisi seed average perimeter =", urgup_perim_avg)

### What are the features of the most narrow seed of each pumpkin seed?

In [ ]:
# Cercevelik min minor axis
cercevelik_miax_min = cercevelik_seeds['Minor_Axis_Length'].min()
# Index
cercevelik_miax_min_idx = cercevelik_seeds['Minor_Axis_Length'].idxmin()

print("Value of most narrow Cercevelik seed =", cercevelik_miax_min) # Value
cercevelik_seeds.iloc[[cercevelik_miax_min_idx]] # Instance

In [ ]:
# Urgup sivrisi min minor axis
urgup_miax_min = urgup_seeds['Minor_Axis_Length'].min()
# Index
urgup_miax_min_idx = urgup_seeds['Minor_Axis_Length'].idxmin()

print("Value of most narrow Urgup Sivrisi seed =", urgup_miax_min) # Value
urgup_seeds.iloc[[urgup_miax_min_idx]] # Instance

### The widest?

In [ ]:
# Cercevelik max minor axis
cercevelik_miax_max = cercevelik_seeds['Minor_Axis_Length'].max()
# Index
cercevelik_miax_max_idx = cercevelik_seeds['Minor_Axis_Length'].idxmax()

print("Value of widest Cercevelik seed =", cercevelik_miax_max) # Value
cercevelik_seeds.iloc[[cercevelik_miax_max_idx]] # Instance

In [ ]:
# Urgup sivrisi max minor axis
urgup_miax_max = urgup_seeds['Minor_Axis_Length'].max()
# Index
urgup_miax_max_idx = urgup_seeds['Minor_Axis_Length'].idxmax()

print("Value of widest Urgup Sivrisi seed =", urgup_miax_max) # Value
urgup_seeds.iloc[[urgup_miax_max_idx]] # Instance

### What is the average width of each of the pumpkin seeds?

In [ ]:
cercevelik_miax_avg = cercevelik_seeds['Minor_Axis_Length'].mean()

print("Cercevelik seed average Minor Axis Length or Width =", cercevelik_miax_avg)

In [ ]:
urgup_miax_avg = urgup_seeds['Minor_Axis_Length'].mean()

print("Urgup Sivrisi seed average Minor Axis Length or Width =", urgup_miax_avg)

### What are the features of the shortest seed of each pumpkin seed?

In [ ]:
# Cercevelik min major axis
cercevelik_majax_min = cercevelik_seeds['Major_Axis_Length'].min()
# Index
cercevelik_majax_min_idx = cercevelik_seeds['Major_Axis_Length'].idxmin()

print("Value of shortest Cercevelik seed =", cercevelik_majax_min) # Value
cercevelik_seeds.iloc[[cercevelik_majax_min_idx]] # Instance

In [ ]:
# Urgup Sivrisi min major axis
urgup_majax_min = urgup_seeds['Major_Axis_Length'].min()
# Index
urgup_majax_min_idx = urgup_seeds['Major_Axis_Length'].idxmin()

print("Value of shortest Urgup Sivrisi seed =", urgup_majax_min) # Value
urgup_seeds.iloc[[urgup_majax_min_idx]] # Instance

### The longest?

In [ ]:
# Cercevelik max major axis
cercevelik_majax_max = cercevelik_seeds['Major_Axis_Length'].max()
# Index
cercevelik_majax_max_idx = cercevelik_seeds['Major_Axis_Length'].idxmax()

print("Value of longest Cercevelik seed =", cercevelik_majax_max) # Value
cercevelik_seeds.iloc[[cercevelik_majax_max_idx]] # Instance

In [ ]:
# Urgup Sivrisi max major axis
urgup_majax_max = urgup_seeds['Major_Axis_Length'].max()
# Index
urgup_majax_max_idx = urgup_seeds['Major_Axis_Length'].idxmax()

print("Value of longest Urgup Sivrisi seed =", urgup_majax_max) # Value
urgup_seeds.iloc[[urgup_majax_max_idx]] # Instance

### What is the average length of each of the pumpkin seeds?

In [ ]:
cercevelik_majax_avg = cercevelik_seeds['Major_Axis_Length'].mean()

print("Cercevelik seed average Major Axis Length or Length =", cercevelik_majax_avg)

In [ ]:
urgup_majax_avg = urgup_seeds['Major_Axis_Length'].mean()

print("Urgup Sivrisi seed average Major Axis Length or Length =", urgup_majax_avg)

The following few questions are questions created for probing on the *Shape** of the seeds. The features that we think are relevant to learning about the difference in shape between the two pumpkin seed species are `Eccentricity`, `Roundness`, and `Convex Area`.

### What are the features of the most circular seed for each of the pumpkin seed species?

In [ ]:
# Cercevelik min eccentricity
cercevelik_ecce_min = cercevelik_seeds['Eccentricity'].min()
# Index
cercevelik_ecce_min_idx = cercevelik_seeds['Eccentricity'].idxmin()

print("Value of most circular Cercevelik seed =", cercevelik_ecce_min) # Value
cercevelik_seeds.iloc[[cercevelik_ecce_min_idx]] # Instance

In [ ]:
# Urgup sivrisi min eccentricity
urgup_ecce_min = urgup_seeds['Eccentricity'].min()
# Index
urgup_ecce_min_idx = urgup_seeds['Eccentricity'].idxmin()

print("Value of most circular Urgup Sivrisi seed =", urgup_ecce_min) # Value
urgup_seeds.iloc[[urgup_ecce_min_idx]] # Instance

### The most elongated?

In [ ]:
# Cercevelik max eccentricity
cercevelik_ecce_max = cercevelik_seeds['Eccentricity'].max()
# Index
cercevelik_ecce_max_idx = cercevelik_seeds['Eccentricity'].idxmax()

print("Value of most elongated Cercevelik seed =", cercevelik_ecce_max) # Value
cercevelik_seeds.iloc[[cercevelik_ecce_max_idx]] # Instance

In [ ]:
# Urgup sivrisi max eccentricity
urgup_ecce_max = urgup_seeds['Eccentricity'].max()
# Index
urgup_ecce_max_idx = urgup_seeds['Eccentricity'].idxmax()

print("Value of most elongated Urgup Sivrisi seed =", urgup_ecce_max) # Value
urgup_seeds.iloc[[urgup_ecce_max_idx]] # Instance

### What is the average eccentricity of each of the pumpkin seed species?

In [ ]:
cercevelik_ecce_avg = cercevelik_seeds['Eccentricity'].mean()

print("Cercevelik seed average eccentricity =", cercevelik_ecce_avg)

In [ ]:
urgup_ecce_avg = urgup_seeds['Eccentricity'].mean()

print("Urgup Sivrisi seed average eccentricity =", urgup_ecce_avg)

### What are the features of the least round seed for each of the pumpkin seed species?

In [ ]:
# Cercevelik min roundness
cercevelik_round_min = cercevelik_seeds['Roundness'].min()
# Index
cercevelik_round_min_idx = cercevelik_seeds['Roundness'].idxmin()

print("Value of least round Cercevelik seed =", cercevelik_round_min) # Value
cercevelik_seeds.iloc[[cercevelik_round_min_idx]] # Instance

In [ ]:
# Urgup sivrisi min roundness
urgup_round_min = urgup_seeds['Roundness'].min()
# Index
urgup_round_min_idx = urgup_seeds['Roundness'].idxmin()

print("Value of least round Urgup Sivrisi seed =", urgup_round_min) # Value
urgup_seeds.iloc[[urgup_round_min_idx]] # Instance

### The most round?

In [ ]:
# Cercevelik max roundness
cercevelik_round_max = cercevelik_seeds['Roundness'].max()
# Index
cercevelik_round_max_idx = cercevelik_seeds['Roundness'].idxmax()

print("Value of least round Cercevelik seed =", cercevelik_round_max) # Value
cercevelik_seeds.iloc[[cercevelik_round_max_idx]] # Instance

In [ ]:
# Urgup sivrisi max roundness
urgup_round_max = urgup_seeds['Roundness'].max()
# Index
urgup_round_max_idx = urgup_seeds['Roundness'].idxmax()

print("Value of least round Urgup Sivrisi seed =", urgup_round_max) # Value
urgup_seeds.iloc[[urgup_round_max_idx]] # Instance

### What is the average roundness of each of the pumpkin seed species?

In [ ]:
cercevelik_round_avg = cercevelik_seeds['Roundness'].mean()

print("Cercevelik seed average roundness =", cercevelik_round_avg)

In [ ]:
urgup_round_avg = urgup_seeds['Roundness'].mean()

print("Urgup Sivrisi seed average roundness =", urgup_round_avg)

**=========================================================================================================================================**

### Correlation Checking

Let’s play around with the data and find association among them. First, we check the correlation between features and labels.

In [ ]:
# temp_df['Class'] = label_encoder.fit_transform(temp_df['Class'])
# temp_df.corr()['Class'].sort_values()

As shown above, the four features which have strongest relationship with Class are <b>Aspect_Ratio, Eccentricity, Major_Axis_Length and Perimeter</b>.

Then we display the correlations of each combination of two features.

In [ ]:
# corr = temp_df.corr().round(2)
# sns.heatmap(corr,cmap="rocket",annot=True)

The brighter the color is, the stronger the relationship between 2 variables.<br>
Notably, 3 features have perfect positive correlation with each other: `Area`, `Convex_Area` and `Equiv_Diameter`. Since these features have almost the same correlations with `Class`, these features could be dropped from the training dataset.<br>
Other closely correlated features are `Aspect_Ratio` and `Eccentricity` with $0.95$ correlation, then `Perimeter` with `Area`, `Convex_Area` and `Equiv_Diameter` with $0.93$ correlation. Meanwhile, `Compactness` and `Aspect_Ratio` are highly inversely correlated at $-0.99$ correlation, implying `Compactness` decreases with increasing `Aspect_Ratio`.

Let’s plot some interesting pattern.

### The number of data in each class

In [ ]:
# g = sns.catplot(data=temp_df,x='Class',kind='count')
# g.set_axis_labels("", "Number of seeds")
# g.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

We can see that in the dataset, the number of instances classified as Çerçevelik seeds is slightly more than that of Ürgüp Sivrisi.

### Boxplot

We display the relationship between Class and the first four features which have strongest relationship with it.

In [ ]:
# # Boxplot
# f = plt.figure(figsize=(12,8))

# plt.subplot(2,2,1)
# # Aspect_Ration vs Class
# a=sns.boxplot(data=temp_df,x='Class',y='Aspect_Ratio')
# a.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

# plt.subplot(2,2,2)
# # Eccentricity vs Class
# b=sns.boxplot(data=temp_df,x='Class',y='Eccentricity')
# b.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

# plt.subplot(2,2,3)
# # Major_Axis_Length vs Class
# c=sns.boxplot(data=temp_df,x='Class',y='Major_Axis_Length')
# c.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

# plt.subplot(2,2,4)
# # Perimeter vs Class
# d=sns.boxplot(data=temp_df,x='Class',y='Perimeter')
# d.set_xticklabels(["Cercevelik", "Urgup Sivrisi"])

Ürgüp Sivrisi has a higher median in all 4 features. Since these features are related to shape and size, this may imply that Ürgüp Sivrisi seeds are generally bigger and more elongated than Çerçevelik seeds.

### Scatterplot

As we can see from the correlation plot, some other combinations of the variables also show strong relationships (around 0.95). Let’s have a look at the four of them.

In [ ]:
# #The relationships among other features
# f = plt.figure(figsize=(16,12))
# #Roundness vs. Compactness
# plt.subplot(2,2,1)
# sns.scatterplot(data=temp_df,x='Compactness', y='Roundness',hue='Class')
# plt.grid()

# #Perimeter vs. Major_Axis_Length
# plt.subplot(2,2,2)
# sns.scatterplot(data=temp_df,x='Perimeter', y='Major_Axis_Length',hue='Class')
# plt.grid()

# #Perimeter vs. Area
# plt.subplot(2,2,3)
# sns.scatterplot(data=temp_df,x='Perimeter', y='Area',hue='Class')
# plt.grid()

# #Perimeter vs. Convex_Area
# plt.subplot(2,2,4)
# sns.scatterplot(data=temp_df,x='Perimeter', y='Convex_Area',hue='Class')
# plt.grid()

The scatterplot is divided according to their class. As you can see two features got very strong relationships. While they don’t have strong relationships with **Class**, which can be seen from the distribution of orange and blue points, representing two different seed classes. The distribution of two classes doesn’t appear like cluster.

## **6** | **Initial Model Training**

### *Model 1: K-Nearest Neighbors*

As long as the data is normalized and scaled properly, KNN can work well with numerical features. It's also faster with smaller data since it just "memorizes" the training set. Using k-fold cross-validation, the model can evaluate predictions with training data alone, by splitting it to smaller subsets. (3.1. Cross-validation: Evaluating Estimator Performance, n.d.)

<b>Setting up Libraries and Datasets</b>

First, we import the needed libraries for KNN model training.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score # Evaluation metrics for later analysis

We'll split the data to training set (70%) and testing set (30%). The training set will be used to train the model while the testing set will be used for evaluation of the model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=1)
print("Training Features (X_train):\n", X_train)
print("Training Labels (y_train):\n", y_train)
print("Testing Features (X_test):\n", X_test)
print("Testing Labels (y_test):\n", y_test)

<b>Hyperparameter Tuning with Cross-Validation</b>

In training the model, we'll need to find a good value for the hyperparameter `k`, the number of neighbors.
We'll test values from 1-20 and get the average cross-validation accuracy of the model on each `k`. The accuracy will be stored in `accuracy_scores`.

In [ ]:
k_choices = range(1, 21)
accuracy_scores = []
for k in k_choices:
    knn = KNeighborsClassifier(n_neighbors=k) # Instantiate
    score = cross_val_score(knn, X_train, y_train, cv=10).mean() # Get the avg cross-val accuracy
    accuracy_scores.append(score)

Let's plot the accuracy scores of each k.

In [ ]:
for i in range(len(accuracy_scores)):
    plt.scatter(k_choices[i], accuracy_scores[i])
plt.xlabel("k")
plt.ylabel("Cross-validation accuracy")
plt.title("Cross-validation on k")
plt.grid()
plt.show()

The best `k` seems to be between $12.5$ and $15.0$, with a cross-validation accuracy higher than 0.87. Now let's compute this based on the accuracy scores.

In [ ]:
best_k = k_choices[np.argmax(accuracy_scores)]
print("Best k:", best_k)

Let's train the model using the best `k` ($14$).

In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_k) # Instantiate with best k
knn.fit(X_train, y_train) # Train

Let's test the model on the testing data.

In [ ]:
y_pred = knn.predict(X_test)
y_pred

### *Model 2: Decision Tree*

Importing the libraries needed for decision tree classifier -> **TO MOVE TO SECTION 3**

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

As we have a classification problem in our hands, we will be using the `DecisionTreeClassifier` library from `sklearn` for training our decision tree model on the pumpkin seeds dataset.

<span style="color:red">why decision tree?</span>

In the following cell, we will be splitting our train (70%) and test (30%) data. It is enough to use the original, non-normalized data for decision trees as normalization does not affect the performance of the model.

*On data normalization:* <br> https://forecastegy.com/posts/do-decision-trees-need-feature-scaling-or-normalization/ <br>
https://sebastianraschka.com/faq/docs/when-to-standardize.html

In [ ]:
dt_X_train, dt_X_test, dt_y_train, dt_y_test = train_test_split(X,y,test_size=0.3,random_state=1)
# print("X train: \n" + str(X_train))
# print("y train: \n" + str(y_train))
# print("X test: \n" + str(X_test))
# print("y test: \n" + str(y_test))

In [ ]:
print("X_train shape : ", dt_X_train.shape)
print("y_train shape : ", dt_y_train.shape)
print("X_test shape : ", dt_X_test.shape)
print("y_test shape : ", dt_y_test.shape)

After splitting, we end up with training data that has 1750 instances and test data with 750 instances.

For this part of the decision tree modeling, we will be using the default parameters for the `DecisionTreeClassifier` to see its baseline performance. In the following cell, we will be defining our decision tree as `pumpkin_dt`.

In [ ]:
pumpkin_dt = DecisionTreeClassifier()

We will be fitting the decision tree model on the training data and creating predictions on the test data in the following cells.

In [ ]:
# Fit the model
pumpkin_dt.fit(dt_X_train, dt_y_train)

In [ ]:
# Test predictions
pumpkin_dt_test_preds = pumpkin_dt.predict(dt_X_test)
pumpkin_dt_test_preds

Now that we have fitted the decision tree model and created predictions on our test data, we can describe and visualize our tree.

The following cell contains the definition of the function for describing the decisiont tree. The tree structure description is taken from this page in scikit-learn: <br> https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py

In [ ]:
def describe_tree(clf):
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold
    values = clf.tree_.value

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        # `pop` ensures each node is only visited once
        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        # If the left and right child of a node is not the same we have a split
        # node
        is_split_node = children_left[node_id] != children_right[node_id]
        # If a split node, append left and right children and depth to `stack`
        # so we can loop through them
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    print(
        "The binary tree structure has {n} nodes and has "
        "the following tree structure:\n".format(n=n_nodes)
    )
    for i in range(n_nodes):
        if is_leaves[i]:
            print(
                "{space}node={node} is a leaf node, values: {values}.".format(
                    space=node_depth[i] * "\t", node=i, values=values[i]
                )
            )
        else:
            print(
                "{space}node={node} is a split node: "
                "go to node {left} if X[:, {feature}] <= {threshold} "
                "else to node {right}.".format(
                    space=node_depth[i] * "\t",
                    node=i,
                    left=children_left[i],
                    feature=feature[i],
                    threshold=threshold[i],
                    right=children_right[i],
                )
            )

In [ ]:
describe_tree(pumpkin_dt)

We ended up getting a tree structure for our test data that has 367 nodes in total. At the root node, it looks at the feature at index 6 to ask the first question, and this feature is the `Eccentricity` of the pumpkin seed.

We can also plot our tree as a graph by calling the `plot_tree` function from `sklearn.tree`.

In [ ]:
tree.plot_tree(pumpkin_dt)
plt.show()

We can get the confusion matrix, classification report, and the test accuracy for the tree now that we have visualized it.

In [ ]:
# Confusion matrix of the model
print("Confusion Matrix:")
print(confusion_matrix(dt_y_test, pumpkin_dt_test_preds))

Note that Çerçevelik is encoded as 0 and Ürgüp Sivrisi as 1. The confusion matrix resulted in the following values:
- **True Positives:** 310 instances were labeled correctly as Urgup Sivrisi.
- **True Negatives:** 305 instances were labeled correctly as Cercevelik.
- **False Negatives:** 61 instances were labeled incorrectly as Cercevelik.
- **False Positives:** 74 instances were labeled incorrectly as Urgup Sivrisi.

From the classification report, we can get the evaluation metrics of the model. We can also get the accuracy of the model by separately calling the `accuracy_score()` function.

In [ ]:
print("Classification Report:")
print(classification_report(dt_y_test, pumpkin_dt_test_preds, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(dt_y_test, pumpkin_dt_test_preds))

The baseline default model resulted in a test accuracy of `0.828` for this dataset. We can improve this through hyperparameter tuning.

<span style="color:red">*What values to consider for each hyperparameter to tune? What basis*</span>

Some regularization techniques for decision trees are deciding on the stopping criterion. We can modify the parameters of the `DecisionTreeClassifier()` model so that our model stops asking questions once it reaches certain thresholds. For this model, we can tune the following parameters: `criterion`, `min_samples_split`, `max_depth`, and `max_leaf_nodes`.

In [ ]:
# Hyperparameters to tune: criterion, min_samples_split, max_depth, max_leaf_nodes
# Values considered for hyperparams can still be changed based on related works found
hyperparameters = [
    {
        "criterion" : ["gini", "entropy"],
        "min_samples_split" : [10,20,50,100,500,1000,2000],
        "max_depth" : [10,20,50,100,500],
        "max_leaf_nodes" : [10,20,50,100,500,1000,1500]
    }
]

Through the `RandomizedSearchCV()`, we can find the best hyperparameters using cross-validation. Since we've defined our hyperparameter options in the previous cell, we can just input this as our parameter for `param_distributions` using 5-fold cross-validation. <br>
*Optionally: explain why RandomizedSearchCV over other search algorithms for determining the best hyperparameters* <br>
*Some sources: https://insidelearningmachines.com/tune_hyperparameters_in_decision_trees/*

In [ ]:
# n_iter: number of parameter settings to be sampled
# cv: number of cross validation folds
rsc_pumpkin = RandomizedSearchCV(estimator=pumpkin_dt,param_distributions=hyperparameters,n_iter=100,cv=5)

We can now fit our data to this `RandomizedSearchCV` model, and it finds the best parameters for us.

In [ ]:
rsc_pumpkin.fit(dt_X_train,dt_y_train)

By calling the `best_params_` variable, we can get the dictionary of best hyperparameter values for our tuned decision tree model.

In [ ]:
rsc_pumpkin.best_params_

Now that we have the best hyperparameters based from `RandomizedSearchCV`, we can define a new estimator model based on these hyperparameters. We follow the same pipeline as the baseline model, in which we fit the model using our train data and create predictions on our test data.

In [ ]:
pumpkin_dt_tuned = DecisionTreeClassifier(min_samples_split=100, max_leaf_nodes=10, max_depth=20,criterion="gini")

In [ ]:
pumpkin_dt_tuned.fit(dt_X_train,dt_y_train)

In [ ]:
pumpkin_dt_tuned_preds = pumpkin_dt_tuned.predict(dt_X_test)
pumpkin_dt_tuned_preds

Now that we have fitted the tuned decision tree model and created predictions on our test data, we can describe and visualize this tuned tree.

In [ ]:
describe_tree(pumpkin_dt_tuned)

In [ ]:
tree.plot_tree(pumpkin_dt_tuned)
plt.show()

We significantly ended up with lesser nodes and a more shallow depth of tree. Our tuned tree now has 19 nodes, and at the root node, it looks at the feature at index 6 to ask the first question. This is the same as our original tree, and it asks about the `Eccentricity` of the pumpkin seed.

### *Model 3: Logistic Regression*

Since the dataset mainly consists of 2 classes ("Cercevelik" and "Urgup Sivrisi"), we can use Logistic Regression in predicting the pumpkin seed class based on the provided features.  This is primarily because Logistic Regression excels in binary classification problems.  The dataset's features, including area, perimeter, major axis lengths, and others, provide valuable information for predicting seed class.

We will use `sklearn`'s `SGDClassifier` to create our logistic regression model. A binomial logistic regression is optimal for our model since there are only two classes (`Çerçevelik` or `Ürgüp Sivrisi`) in our dataset.

We will split the data into 70% for the training set and 30% for the testing set.

We will use the normalized data to ensure reliable and efficient SGDClassifier performance.

In [ ]:
logreg_X_train, logreg_X_test, logreg_y_train, logreg_y_test = train_test_split(X_norm, y, test_size=0.3, random_state=1)
print("X_train shape : ", logreg_X_train.shape)
print("y_train shape : ", logreg_y_train.shape)
print("X_test shape : ", logreg_X_test.shape)
print("y_test shape : ", logreg_y_test.shape)

Visualize the train data.

In [ ]:
plt.scatter(logreg_X_train[:, 0], logreg_X_train[:, 1], c=logreg_y_train)
plt.title('Train data')

Visualize the test data.

In [ ]:
plt.scatter(logreg_X_test[:, 0], logreg_X_test[:, 1], c=logreg_y_test)
plt.title('Test data')

Import the `SGDClassifier` class.

In [ ]:
from sklearn.linear_model import SGDClassifier

**Setting `SGDClassifier` parameters**

| Name                      | Parameter       | Value      | Description                                                                                                                     |
|---------------------------|-----------------|------------|---------------------------------------------------------------------------------------------------------------------------------|
| **Loss function**         | `loss`          | 'log_loss' | Set to `'log_loss'` for logistic regression.                                                                                    |
| **Initial learning rate** | `eta0`          | 0.001      | Set the initial learning rate to a small value (`0.001`) to ensure gradual updates, preventing overshooting optimal parameters. |
| **Maximum iterations**    | `max_iter`      | 200        | Limit the training iterations to `200` to prevent overfitting.                                                                  |
| **Learning rate**         | `learning_rate` | 'constant' | Maintain a constant learning rate throughout training.                                                                          |
| **Random state**          | `random_state`  | 1          | Ensure reproducibility by fixing the random seed.                                                                               |
| **Verbose**               | `verbose`       | 1          | Display training progress.                                                                                                      |

In [ ]:
logreg = SGDClassifier(
    loss='log_loss',
    eta0=0.001,
    max_iter=200,
    learning_rate='constant',
    random_state=1,
    verbose=1,
)

Train the model by calling the `fit()` function

In [ ]:
logreg.fit(logreg_X_train, logreg_y_train)

To see if our model does well, we test our trained model on the test set and get the prediction results.


In [ ]:
logreg_pred = logreg.predict(logreg_X_test)
print(logreg_pred)

## **7** | **Error Analysis**

### *Model 1: K-Nearest Neighbors*

Let's evaluate the KNN model. First, let's train the KNN model again, with the same configurations in the last section.

In [ ]:
accuracy_scores = []
k_choices = range(1, 21)
for k in k_choices:
    knn = KNeighborsClassifier(n_neighbors=k) # Instantiate
    score = cross_val_score(knn, X_train, y_train, cv=10).mean() # Get the avg cross-val accuracy
    accuracy_scores.append(score)
# Train model with best k
best_k = k_choices[np.argmax(accuracy_scores)]
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Note that Çerçevelik is encoded as `0` and Ürgüp Sivrisi as `1`.  
<b>True positives:</b> $306$ instances were labeled correctly as Ürgüp Sivrisi.<br>
<b>True negatives:</b> $346$ instances were labeled correctly as Çerçevelik.<br>
<b>False negatives:</b> $65$ instances were labeled incorrectly as Çerçevelik.<br>
<b>False positives:</b> $33$ instances were labeled incorrectly as Ürgüp Sivrisi.<br>

Given the higher amount of true positives and false negatives, it seems that Ürgüp Sivrisi seeds are harder to classify correctly. Let's get the evaluation metrics per class using `classification_report()`, and then the overall test accuracy.

In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(y_test, y_pred))

KNN model gives a 0.873 test accuracy.

### *Model 2: Decision Trees*

We can now get the confusion matrix of the results of the tuned Decision Tree model.

In [ ]:
# Confusion matrix of the model
print("Confusion Matrix:")
print(confusion_matrix(dt_y_test, pumpkin_dt_tuned_preds))

Note that Çerçevelik is encoded as `0` and Ürgüp Sivrisi as `1`. The confusion matrix of the tuned tree resulted in the following values:
- **True Positives:** 330 instances were labeled correctly as Urgup Sivrisi.
- **True Negatives:** 321 instances were labeled correctly as Cercevelik.
- **False Negatives:** 41 instances were labeled incorrectly as Cercevelik.
- **False Positives:** 58 instances were labeled incorrectly as Urgup Sivrisi.

From the classification report, we can get the evaluation metrics of the tuned model. We can also get its accuracy by separately calling the `accuracy_score()` function.

In [ ]:
print("Classification Report:")
print(classification_report(y_test, pumpkin_dt_tuned_preds, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(y_test, pumpkin_dt_tuned_preds))

In the Decision Tree model, the **Çerçevelik** class resulted in a precision of **0.89** and a recall of **0.85**, while the **Ürgüp Sivrisi** class resulted in a precision of **0.85** and a recall of **0.89**.

The tuned model resulted in a test accuracy of `0.868` for this dataset.

### *Model 3: Logistic Regression*

Print the confusion matrix to evaluate the performance of the Logistic Regression model. 

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(logreg_y_test, logreg_pred))

Note that Çerçevelik is encoded as `0` and Ürgüp Sivrisi as `1`. 
- **True positives**: 301 instances were labeled correctly as Ürgüp Sivrisi
- **True negatives**: 335 instances were labeled correctly as Çerçevelik
- **False positives**: 44 instances were labeled incorrectly as Ürgüp Sivrisi
- **False negatives**: 70 instances were labeled incorrectly as Çerçevelik

Print the precision, recall, and other evaluation metrics per class using `classification_report()`.

In [ ]:
print("Classification Report:")
print(classification_report(logreg_y_test, logreg_pred, target_names=label_encoder.classes_))

The **Çerçevelik** class has a precision of **0.83** and a recall of **0.88**, while the **Ürgüp Sivrisi** class has a precision of **0.87** and a recall of **0.81**.

Print the accuracy of the Logistic Regression model.

In [ ]:
print("Test Accuracy:", accuracy_score(logreg_y_test, logreg_pred))

The Logistic Regression model has an accuracy of **0.848**.

## **8** | **Improving Model Performance**

### *Model 1: K-Nearest Neighbors*

To improve the performance of KNN, let's use Grid Search to get the best combination of hyperparameters `k`, `weights` and `distance metric`. First, we'll import the needed library for Grid Search.

In [ ]:
from sklearn.model_selection import GridSearchCV

We'll set the parameter grid with a slightly increased range of `k`, the distance metrics as `euclidean`, `manhattan`, and `minkowski`, and the weights as `uniform` and `distance`.
<ul>
<li>k - number of neighbors KNN will consider</li>
<li>weights - determines if closer neighbors have more influence</li>
<li>distance metric - determines how distance is calculated to consider neighbors</li>
</ul>

In [ ]:
knn = KNeighborsClassifier() # Instantiate

param_grid = {
    'n_neighbors': range(1,31), # values for k, slightly increased
    'metric': ['euclidean', 'manhattan', 'minkowski'], # distance metrics
    'weights': ['uniform', 'distance']
}

We'll also perform grid search with slightly more k-folds.

In [ ]:
grid_search = GridSearchCV(knn, param_grid, cv=20, scoring='accuracy') # Instantiate
grid_search.fit(X_train, y_train)

Let's look at the results of different hyperparameter configurations.

In [ ]:
pd.DataFrame(grid_search.cv_results_).sort_values(by=["rank_test_score"])

Based on the top results, the best hyperparameter configurations are generally:
<ul>
<li>k - mixed, mostly 12</li>
<li>weights - mostly distance</li>
<li>distance metric - mixed, but mostly manhattan</li>
</ul>
Let's get the actual best configuration an cross-validation accuracy.

In [ ]:
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-validation Accuracy:", grid_search.best_score_)

Next, let's try using Random Search with similar configurations used in Grid Search.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(knn, param_grid, n_iter=50, cv=10, random_state=1, scoring='accuracy')
random_search.fit(X_train, y_train)

print("Best Hyperparameters:", random_search.best_params_)
print("Best Cross-validation Accuracy:", random_search.best_score_)

Grid Search has a slightly higher cross-validation accuracy than Random Search so let's use grid search moving forward. Now, we'll test the model with the best hyperparameter configuration.

In [ ]:
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)
y_pred

Let's evaluate the model by printing out some evaluation metrics again.

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(y_test, y_pred))

The test accuracy improved from 0.873 to 0.877.

### *Model 2: Decision Trees*

Import the `HistGradientBoostingClassifier` class.

In [ ]:
# HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

Create an instance of `HistGradientBoostingClassifier` using its default parameters.

The parameters and its default value can be found in the documentation: https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html

In [ ]:
pumpkin_boosting = HistGradientBoostingClassifier()

Train the model on our training data.

In [ ]:
# fit
pumpkin_boosting.fit(dt_X_train, dt_y_train)

Get the prediction results on the test data.

In [ ]:
pumpkin_boosting_test_preds = pumpkin_boosting.predict(dt_X_test)

pumpkin_boosting_test_preds

Print the evaluation metrics of the model to evaluate its performance.

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(dt_y_test, pumpkin_boosting_test_preds))

print("Classification Report:")
print(classification_report(dt_y_test, pumpkin_boosting_test_preds, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(dt_y_test, pumpkin_boosting_test_preds))

Note that Çerçevelik is encoded as `0` and Ürgüp Sivrisi as `1`. 
- **True positives**: 326 instances were labeled correctly as Ürgüp Sivrisi
- **True negatives**: 341 instances were labeled correctly as Çerçevelik
- **False positives**: 38 instances were labeled incorrectly as Ürgüp Sivrisi
- **False negatives**: 45 instances were labeled incorrectly as Çerçevelik

The **Çerçevelik** class has a precision of **0.88** and a recall of **0.90**, while the **Ürgüp Sivrisi** class has a precision of **0.90** and a recall of **0.88**.

The accuracy of the model is `0.8893`.

RandomSearchCV using hyperparameters of DT  
*Resources:* https://www.restack.io/p/hyperparameter-tuning-answer-histgradientboostingclassifier-tuning-cat-ai

Specify the hyperparameters and their ranges for `HistGradientBoostingClassifier`.

In [ ]:
hyperparameters_boosting = [
    {
        "learning_rate": [0.01,0.05,0.1],
        "max_iter": [100, 200, 300, 400, 500],
        "max_depth": [10,20,50,100,500]
    }
]

Tune the hyperparameters by performing randomized search (`RandomizedSearchCV`) using 5-fold cross-validation.

In [ ]:
rsc_pumpkin_boosting = RandomizedSearchCV(estimator=pumpkin_boosting,param_distributions=hyperparameters_boosting,n_iter=50,cv=5)

Train the models on our training data.

In [ ]:
rsc_pumpkin_boosting.fit(dt_X_train,dt_y_train)

Find the best hyperparameters found by the randomized search.

In [ ]:
rsc_pumpkin_boosting.best_params_

Create a new estimator based on the best hyperparameters found by `RandomizedSearchCV`.

In [ ]:
pumpkin_boosting_tuned = HistGradientBoostingClassifier(max_iter=400,max_depth=10,learning_rate=0.01)

In [ ]:
pumpkin_boosting_tuned.fit(dt_X_train,dt_y_train)

In [ ]:
pumpkin_boosting_tuned_preds = pumpkin_boosting_tuned.predict(dt_X_test)

pumpkin_boosting_tuned_preds

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(dt_y_test, pumpkin_boosting_tuned_preds))

print("Classification Report:")
print(classification_report(dt_y_test, pumpkin_boosting_tuned_preds, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(dt_y_test, pumpkin_boosting_tuned_preds))

Test accuracy of the tuned HistGradientBoostingClassifier resulted in `0.888`, which does not seem to be much of an improvement as there is only `0.02` difference in accuracy from the actual tuned decision tree model.

### *Model 3: Logistic Regression*

Get the parameters of `SGDClassifier`.

In [ ]:
logreg.get_params().keys()

We will mainly focus on tweaking the `'eta0'`, `'max_iter'`, and `'learning_rate'` parameters.

The possible values of each parameter can be found in the documentation:
https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.SGDClassifier.html

Specify the hyperparameters and their ranges.

In [ ]:
logreg_params = {
    'eta0': [0.001, 0.01, 0.1],
    'max_iter': [100, 200, 500],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
}

We'll perform hyperparameter tuning using `GridSearchCV`.

Set the cross-validation (`cv`) parameter to `10` to improve accuracy by providing more precise estimates.

In [ ]:
logreg_gs = GridSearchCV(logreg, logreg_params, cv=10, scoring='accuracy')

Train our models on our training data.

In [ ]:
logreg_gs.fit(logreg_X_train, logreg_y_train)

Find the best hyperparameters found by the grid search.

In [ ]:
logreg_gs.best_params_

Print the best score of the grid search.

In [ ]:
logreg_gs.best_score_

Now, let's try conducting hyperparameter tuning using `RandomizedSearchCV` to see if it's better than `GridSearchCV`.

We also set the cross-validation (`cv`) to `10`, similar to above.

We will set the number of iterations (`n_iter`) parameter to be equal to the number of total possible combinations of hyperparameters.  In this case, since there are $3$ values for `eta0`, $3$ values for `max_iter`, and $4$ values for `learning_rate`, then `n_iter` should be set to $3 \times 3 \times 4 = 36$.

In [ ]:
logreg_rs = RandomizedSearchCV(
    logreg,
    logreg_params,
    n_iter=36,
    cv=10,
    random_state=1,
    scoring='accuracy'
)

Train our models on our training data.

In [ ]:
logreg_rs.fit(logreg_X_train, logreg_y_train)

Find the best hyperparameters found by the randomized search.

In [ ]:
logreg_rs.best_params_


Print the best score of the grid search.

In [ ]:
logreg_rs.best_score_

Since `GridSearchCV` and `RandomizedSearchCV` both found the same best hyperparameters and both also got the same best score, we will just use the hyperparameters of `GridSearchCV` in making an estimator.

Create a new estimator based on the best hyperparameters found by `GridSearchCV`.

In [ ]:
logreg_best = SGDClassifier(
    loss='log_loss',
    eta0=logreg_rs.best_params_['eta0'],
    max_iter=logreg_rs.best_params_['max_iter'],
    learning_rate=logreg_rs.best_params_['learning_rate'],
    random_state=1,
    verbose=1
)

Train the estimator on our data.

In [ ]:
logreg_best.fit(logreg_X_train, logreg_y_train)

Get the prediction results on the test data.

In [ ]:
logreg_best_pred = logreg_best.predict(logreg_X_test)
print(logreg_best_pred)

Evaluate the performance of the best model by printing the evaluation metrics.

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(logreg_y_test, logreg_best_pred))

print("Classification Report:")
print(classification_report(logreg_y_test, logreg_best_pred, target_names=label_encoder.classes_))

print("Test Accuracy:", accuracy_score(logreg_y_test, logreg_best_pred))

In comparison with the initial model, the **accuracy** of the best model increased to `0.864` from `0.848`.

The **precision** of the **Cercevelik** class also increased to `0.85` from `0.83`, and the **recall** of the **Urgup Sivrisi** class increased to `0.85` from `0.81`.

## **9** | **Model Performance Summary**

### *Model 1: K-Nearest Neighbors*

The initial model performance was 0.873 while the improved model performance was 0.877. Based on the Grid Search algorithm, the best configurations for KNN are as follows: 
<ul>
<li>k - 12</li>
<li>weights - distance</li>
<li>distance metric - manhattan</li>
</ul>
Generally, these configurations help decrease sensitivity to outliers. Distance-based weights increase the influence of closer neighbors while manhattan distance metric avoids amplifying distances, which helps resist outliers, opposite to Euclidean distance which squares the distances.(Nixon & Aguado, 2020)

## **10** | **Insights and Conclusions**

The pumpkin seeds dataset's Exploratory Data Analysis (EDA) revealed several key insights that significantly impact the model training process. Firstly, features like *'Aspect_Ration', 'Eccentricity', and 'Major_Axis_Length'* demonstrated strong positive correlations with the target class, while *'Compactness', 'Roundness', and 'Minor_Axis_Length'* exhibited strongly negative correlations. This **variation in correlation strengths provided a rationale for selecting these features for model training**, anticipating that they would offer a better predictive performance.

### Recommendations for Model Improvement:

In the pumpkin seeds dataset, the EDA revealed features with a **level of uniformity and correlation**, contributing to the similar accuracy achieved by *kNN, Logistic Regression, and Decision Trees*. Additionally, the absence of extreme outliers or skewed distributions allowed these diverse models to capture the essential patterns in the data effectively. This scenario led to a convergence in performance, where even simpler models were as effective as more complex ones in predicting class labels. Consequently, these varied modeling approaches sufficiently interpreted the dataset's structure and feature relationships.

The EDA provided invaluable insights into the dataset, guiding feature selection and forming hypotheses about model performance. The combination of EDA, model selection, and hyperparameter tuning was a practical approach, resulting in models that effectively captured the underlying patterns in the data. This comprehensive methodology highlights the importance of thorough exploratory analysis and strategic model optimization in predictive modeling.

## **11** | **References**

1. Koklu, M., Sarigil, S., & Ozbek, O. (2021). The use of machine learning methods in classification
of pumpkin seeds (Cucurbita pepo L.). Genetic Resources and Crop Evolution, 68 (7), 2713-2726.
Doi: https://doi.org/10.1007/s10722-021-01226-0
2. Nixon, M. S., & Aguado, A. S. (2020). Feature extraction and image processing for computer vision (4th ed., pp. 571–604). Elsevier. https://doi.org/10.1016/B978-0-12-814976-8.00012-9
3. 3.1. Cross-validation: evaluating estimator performance. (n.d.). Scikit-learn. https://scikit-learn.org/1.5/modules/cross_validation.html